# Lab 5 - Using `Turing.jl` for Bayesian inference

 **Due**: Monday, 3/20 by 1:00pm.



Make sure you include your name and ID below for submission. <br>
**Name**:  <br>
**ID**:



## Overview

### Description

This lab will introduce you to the syntax and workflow for using `Turing.jl` for Bayesian inference with Julia.

[`Turing.jl`](https://turinglang.org/dev/docs/using-turing/) is a probabilistic programming framework which primarily implements Hamiltonian Monte Carlo, but also allows the use of Random-Walk Metropolis-Hastings (with [`AdvancedMH.jl`](https://github.com/TuringLang/AdvancedMH.jl)) and integrates well with Julia's [Flux](https://fluxml.ai/) machine learning and differentiation packages.

In this lab, you will use `Turing.jl` to fit a model for tide gauge data from [Sewell's Point, Virginia, USA](https://tidesandcurrents.noaa.gov/waterlevels.html?id=8638610) from 2015. This model is not actually a good model, as you will see, but it will introduce you to `Turing`'s syntax and the MCMC workflow.

### Learning Objectives

After completing this lab, students will be able to:

- use `Turing.jl` to fit a simple statistical model using data;
- generate and interpret diagnostic plots for Markov chain Monte Carlo convergence;
- simulate synthetic data for posterior predictive model diagnostics.


## Setup

In [ ]:
# load project environment
import Pkg # load the Pkg package manager
Pkg.activate(@__DIR__) # activate the environment in the directory of the script file
Pkg.instantiate() # make sure all of the needed packages are installed with correct versions

In [ ]:
using Random
using StatsBase
using CSV
using Dates
using DataFrames
using DataFramesMeta
using Distributions
using Plots
using StatsPlots
using LaTeXStrings
using Optim
using Turing

---

## Introduction

As this tutorial involves random number generation, we will set a random
seed to ensure reproducibility.

In [ ]:
Random.seed!(1);

> **Positive Control Tests**
>
> Simulating data with a known data-generating process and then trying to obtain the parameters for that process is an important step in any workflow. We have skipped that here, but a good test is to reproduce this workflow for a known linear regression model.

#### Load Data

Monthly-averaged tide-gauge data for Sewell's Point, VA has been provided in `data/norfolk-monthly-tide-gauge.txt` (from the [Permanent Service for Mean Sea Level](https://psmsl.org/data/obtaining/stations/299.php)). Monthly averages are convenient, because they are approximately the length of a tidal cycle, which is one source of correlation which can impact treating observations as independent. In general, tide gauge data should not be used for statistical analyses at higher frequencies unless one is accounting for the sinusoidal patterns related to tides and seasonal cycles (as NOAA does for its data products).

This dataset consists of several semi-colon-delimited columns, but the first two are the ones that we care about:
- Column 1 is the date, given in a strange decimal format: `year + (month-0.5)/12.0`. We'll want to reformat this into a more useful date-time format.
- Column 2 is the tide gauge measurement in `mm`. The benchmark datum might be slightly different than NOAA's tide gauge record, but all that matters for our purpose is internal consistency.

Let's write a function to load the data.

In [ ]:
function load_data(fname)
    date_format = DateFormat("yyyy-mm")
    # This uses the DataFramesMeta.jl package, which makes it easy to string together commands to load and process data
    df = @chain fname begin
        CSV.File(; delim=';', header=false)
        DataFrame
        rename("Column1" => "date", "Column2" => "gauge")
        # need to reformat the decimal date in the data file
        @transform :year = Int.(floor.(:date))
        @transform :month = Int.(round.((mod.(:date, 1) * 12) .+ 0.5))
        @transform :datetime = Dates.format.(Date.(:year, :month), date_format)
        # replace -99999 with missing
        @transform :gauge = ifelse.(:gauge .== -99999, missing, :gauge)
        select(:datetime, :gauge)
    end
    return df
end

In [ ]:
fname = "data/norfolk-monthly-tide-gauge.txt"
tide_dat = load_data(fname)

Next, let's plot the data.

In [ ]:
# This uses the StatsPlots recipe for plotting DataFrames
@df tide_dat plot(:datetime, :gauge; ylabel="Gauge Measurement (mm)", xlabel="Date", legend=:false, xrotation=30)

## Problems

The goal for this lab is to quantify uncertainty in the mean rate of local sea-level rise (which includes global sea-level rise and local factors such as subsidence, which are quite important in the Norfolk area) over the 1920-2022 period at Sewell's Point. The data above looks roughly linear, so we'll use a simple linear model, assuming that the autocorrelations are independent and identically-distributed:
$$
\begin{align*}
y(t) &= \alpha + \beta t + \varepsilon, \\
\varepsilon &\sim \text{Normal}(0, \sigma^2).
\end{align*}
$$

### Problem 1: Write Turing Model (10 points)

The first task is to write a model in `Turing.jl` format. Models are specified using the `@model` macro, which acts on functions which take data (including auxiliary data, such as covariates) into account. In this case, we don't need any covariates, because our model is just based on a time index.

Priors and likelihoods are specified using the following syntax: `value ~ distribution`, where the distribution uses the syntax from `Distributions.jl`. For example, to specify a $\text{Normal}(0, 5)$ prior for a parameter $\psi$:
```julia
ψ ~ Normal(0, 5)
```
This also makes it convenient to specify truncated distributions, using the `truncated()` function. This is handy for specifying parameters, such as variance parameters such as $\sigma^2$, which can only take positive values.

Note that the order in which the priors are specified matters, as they are sampled in sequence. So, this example (taken from the [documentation](https://turinglang.org/v0.24/docs/using-turing/guide#modelling-syntax-explained)) works:
```julia
s ~ Poisson(1)
y ~ Normal(s, 1)
```
but this does not:
```julia
y ~ Normal(s, 1)
s ~ Poisson(1)
```

For the likelihood, the syntax is similar, but sometimes there is some additional work needed to compute *e.g.* the mean of the distribution. One tip is not to create too many auxiliary variables, as this can slow the model down, or `Turing` will add additional variables. For example (and this isn't relevant here, but is on HW3), you shouldn't do the following:
```julia
model_out = simulation_model(...)
residuals = y .- model_out
for t = 1:length(residuals)
    residuals[t] ~ Normal(0, σ²)
end
```
as it will treat `residuals` as another random variable. Instead, try to specify the likelihood directly in terms of the data, as in:
```julia
model_out = simulation_model(...)
for t = 1:length(y)
    y[t] ~ Normal(model_out[t], σ²)
end
```

There are some other tips in the [`Turing.jl` documentation](https://turinglang.org/dev/docs/using-turing/).

In the code block below, modify the `slr_regression` function to reflect the linear SLR model. You'll need to pick some priors: I don't need you to justify these carefully for this lab, but be thoughtful (and remember that your results can often be sensitive to these priors, so you may want to experiment a bit).  

In [ ]:
@model function slr_regression(y)
    ## specify priors; these are crazy
    σ² ~ truncated(Normal(0, 100); lower=0)
    a ~ Normal(0, 100)
    b ~ Normal(0, 100)

    ## specify likelihood
    # we can specify the likelihood with a loop, calculating the relevant mean at each data point 
    # we could also rewrite this model using linear algebra to compute the joint likelihood, which is often more efficient for large and/or complex models or datasets, but the loop will be more readable in this simple case, and we won't have to worry about numerical instabilities.
    for t = 1:length(y)
        # this adds the (log-)likelihood of each data point to the total
        y[t] ~ Normal(a + b * t, σ²)
    end
end

### Problem 1.2: Sample from the Posterior (5 points)

Next, sample from the posterior using MCMC with the `sample` function. The default sampler used by `Turing.jl` is the [No U-Turn Sampler (NUTS)](https://arxiv.org/abs/1111.4246), which is an adaptive Hamiltonian Monte Carlo sampler. This is a good default, and there are few reasons to use a different one, unless you have an external model which can't be auto-differentiated, and therefore need to use Metropolis-Hastings (with [`AdvancedMH.jl`](https://github.com/TuringLang/AdvancedMH.jl)).

The key parameters that can be set for NUTS are:
- the number of iterations `n_adapts` used to tune the stepsize;
- the target acceptance ratio `δ`.

In this example, we will not set either, and will just use the defaults.

To sample, we need to specify the following:
- the number of iterations per chain;
- the number of chains (for Gelman-Rubin statistics and other convergence heuristics);
- whether to drop the warmup portion of the chain (which is a good idea, since Hamiltonian MC generally requires fewer iterations than M-H, and so these initial portions can have a stronger biasing effect).

The number of chains defaults to 1, but we will usually want to run more for convergence checks. For multi-threaded or sampling, call `MCMCThreads()`, which we will do below (though this is unlikely to actually work since we didn't start this Julia instance [with multiple threads](https://docs.julialang.org/en/v1/manual/parallel-computing/#man-multithreading-1); you can change this in the settings for the Julia VS Code extension for use with notebooks, or by starting Julia with the `-t` flag at startup, *e.g.* `julia --threads 4`). You can also just remove `MCMCThreads()` from the `sample()` function call to ensure that chains are sampled in serial.

It can also be convenient to include all of the sampling configuration in a `let` block, as we do below, to avoid creating additional global variables.

In [ ]:
# There are some missing records, so we'll drop those
# This is ok because we're assuming all of the data are independent residuals. We could also treat them as missing data with some prior or likelihood and Turing would sample accordingly.
dat = dropmissing(tide_dat, :gauge)

In [ ]:
slr_model = slr_regression(dat[!, :gauge])# create the model object with the data

In [ ]:
# call the sampler with 4 chains, 5000 iterations and drop the "burn-in/warm-up" portion
chain = sample(slr_model, NUTS(), MCMCThreads(), 5000, 4, drop_warmup=true);
@show chain

How can we interpret the output? The first parts of the summary
statistics are straightforward: we get the mean, standard deviation, and
Monte Carlo standard error (`mcse`) of each parameter. We also get
information about the effective sample size (ESS) and $\hat{R}$, the Gelman-Rubin statistic.

### Problem 1.3: Evaluate Convergence (10 points)

Plot the chains with `plot(chain)`. You can also make more specific plots with `densityplot`, `histogram`, and `traceplot` (for a full set of available plots, see [the `MCMCChains.jl` documentation](https://docs.juliahub.com/MCMCChains/QRkwo/3.0.12/)). Based on this (and any other visual diagnostics you want to make) and the quantiative diagnostics, do you think the chain has converged? Would you want to run the sampler longer?

### Problem 1.4: Interpreting Results (10 points)

A useful visual representation is a corner plot (which is similar to a pairs plot). You can make a corner plot with `corner(chain)`. Make a corner plot. What can you conclude about the marginal distributions and correlations of the parameters?

You can also find point estimates, such as the maximum-likelihood estimate (MLE) and maximum *a posteriori* estimate (MAP) by loading `Optim.jl`. This works with the following syntax:
```julia
slr_model = slr_regression(data)
optimize(slr_model, MLE()) # or MAP()
```

Find the MLE and MAP estimates. These can sometimes be useful as [starting points for the MCMC sampler](https://turinglang.org/dev/docs/using-turing/guide#sampling-with-the-mapmle-as-initial-states) as they are high-probability states. How do they compare to the full set of posterior samples? What are your thoughts on when you might want to use these estimates versus obtaining and using the full MCMC workflow?

### Problem 1.5: Model Diagnostics and Posterior Predictive Checks (15 points)

A key part of [the Bayesian workflow](https://arxiv.org/abs/2011.01808) is to evaluate the goodness-of-fit of the model. For example, if the posteriors are too narrow, that might suggest that the prior was too restrictive (or it might not! This is just one piece of evidence). This can also guide assessments of the assumptions made when specifying the residual structure and/or the likelihood, or if there were issues with the sampling procedure.

A common category of Bayesian model diagnostics are *posterior predictive checks*, which can be [visual](https://arxiv.org/abs/1709.01449) or [quantitative](http://www.stat.columbia.edu/~gelman/research/published/A6n41.pdf). These allow us to use the generative qualities of a Bayesian model to evaluate whether the posterior predictive distribution $p(\hat{y} | y)$ adequately captures key patterns in the data. For example, we can write a function to simulate pseudo-data from a subsample of the MCMC chain. 

Complete the `predict_slr` function below to generate pseudo-data realizations from the MCMC samples.

In [ ]:
function predict_slr(chain)
    # the Array(group()) syntax is more general than we need, but will work if we have multiple variables which were sampled as a group, for example multiple regression coefficients.
    a = Array(group(chain, :a))
    b = Array(group(chain, :b))
    σ² = Array(group(chain, :σ²))

    # write code here to simulate the model
    y = ...
    return y
end

Now we can generate a predictive interval and median and compare to the
data.

In [ ]:
y_pred = predict_slr(chain)

Notice the dimension of `y_pred`: we have 20,000 columns, because we
have 4 chains with 5,000 samples each. If we had wanted to subsample
(which might be necessary if we had hundreds of thousands or millions of
samples), we could have done that within `predict_slr` before
simulation.

Next, we can get the boundaries of the 95% prediction interval for each year, along with the median. The quantiles are obtained below; plot the median and 95% prediction interval and compare to the original data (by plotting the original data on top of the prediction interval). How does it look?

In [ ]:
# get the boundaries for the 95% prediction interval and the median
y_ci_low = quantile.(eachrow(y_pred), 0.025);
y_ci_hi = quantile.(eachrow(y_pred), 0.975);
y_med = quantile.(eachrow(y_pred), 0.5);

Other visual diagnostics that we could consider include the autocorrelation and histogram of the median residuals, which could identify whether our assumption of independent and identically-distributed normal noise were reasonable. 

Quantitative diagnostics include the *surprise index*, which quantifies the fraction of data points which are outside of a given prediction interval. This is a valuable metric, because an ideally-calibrated model will include $\alpha$% of the data in an $\alpha$-prediction interval.

Now, compute the surprise index for the 95% prediction interval using the quantiles computed above. What can you conclude about how well your model is calibrated? What might you change as a result?